In [3]:
import requests
from pprint import pprint
import pandas as pd
import zipfile
import io
import json
import os
os.makedirs("results/inputs_extracted_xml", exist_ok=True)
year = "2025"

# Source = "https://www.prix-carburants.gouv.fr/rubrique/opendata/" (site gouvernemental)
url = f"https://donnees.roulez-eco.fr/opendata/annee/{year}"

response = requests.get(url)
print(response)

zip_file = zipfile.ZipFile(io.BytesIO(response.content))
file_name = zip_file.namelist()[0]
print(file_name)

with zipfile.ZipFile(io.BytesIO(response.content), 'r') as zip_ref:
    zip_ref.extractall("results/inputs_extracted_xml")
print(file_name, "extracted")

<Response [200]>
PrixCarburants_annuel_2025.xml
PrixCarburants_annuel_2025.xml extracted


In [4]:
import xml.etree.ElementTree as ET
import html

tree = ET.parse(f"results/inputs_extracted_xml/PrixCarburants_annuel_{year}.xml")
root = tree.getroot()

data = []
for pdv in root.findall("pdv"):
    for p in pdv.findall("prix"):
        row = {
            # "id": pdv.get("id"),
            "id": int(pdv.get("id")),
            "latitude": pdv.get("latitude"),
            "longitude": pdv.get("longitude"),
            "cp": int(pdv.get("cp")),
            "ville": pdv.find("ville").text if pdv.find("ville") is not None and pdv.find("ville").text else None,
            # "ville": (pdv.find("ville").text.encode("latin1").decode("utf-8") if pdv.find("ville") is not None and pdv.find("ville").text else None),
            # "adresse": html.unescape(pdv.find("adresse").text).replace(",", " ").replace(";", " ") if pdv.find("adresse") is not None and pdv.find("adresse").text else None,
            "adresse": pdv.find("adresse").text.replace(",", " ").replace(";", " ") if pdv.find("adresse") is not None and pdv.find("adresse").text else None,
            "nom": p.get("nom"),
            "maj": p.get("maj").split("T")[0] if p.get("maj") is not None and "T" in p.get("maj") else None,
            "valeur": float(p.get("valeur")) if p.get("valeur") is not None else None
        }
        data.append(row)

df = pd.DataFrame(data)

os.makedirs("results/inputs_csv", exist_ok=True)
df.to_csv(f"results/inputs_csv/PrixCarburants_annuel_{year}.csv", index=False)
df.head(25)

,id,latitude,longitude,cp,ville,adresse,nom,maj,valeur
0,1000001,4620100,519800,1000,SAINT-DENIS-LèS-BOURG,596 AVENUE DE TREVOUX,Gazole,2025-01-02,1.707
1,1000001,4620100,519800,1000,SAINT-DENIS-LèS-BOURG,596 AVENUE DE TREVOUX,Gazole,2025-01-02,1.707
2,1000001,4620100,519800,1000,SAINT-DENIS-LèS-BOURG,596 AVENUE DE TREVOUX,Gazole,2025-01-03,1.725
3,1000001,4620100,519800,1000,SAINT-DENIS-LèS-BOURG,596 AVENUE DE TREVOUX,Gazole,2025-01-07,1.705
4,1000001,4620100,519800,1000,SAINT-DENIS-LèS-BOURG,596 AVENUE DE TREVOUX,Gazole,2025-01-09,1.688
5,1000001,4620100,519800,1000,SAINT-DENIS-LèS-BOURG,596 AVENUE DE TREVOUX,Gazole,2025-01-10,1.686
6,1000001,4620100,519800,1000,SAINT-DENIS-LèS-BOURG,596 AVENUE DE TREVOUX,Gazole,2025-01-13,1.697
7,1000001,4620100,519800,1000,SAINT-DENIS-LèS-BOURG,596 AVENUE DE TREVOUX,Gazole,2025-01-15,1.747
8,1000001,4620100,519800,1000,SAINT-DENIS-LèS-BOURG,596 AVENUE DE TREVOUX,Gazole,2025-01-17,1.754
9,1000001,4620100,519800,1000,SAINT-DENIS-LèS-BOURG,596 AVENUE DE TREVOUX,Gazole,2025-01-20,1.753


In [ ]:
len(df["id"].unique())

In [ ]:
df.isnull().sum()